In [1]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install PyMuPDF python-docx

Note: you may need to restart the kernel to use updated packages.


RANDOMLY SELECTED TEXT FOR RED CROSSED, HIGHLIGHT AND COMMENTS TO BE ADDED

In [3]:
from docx import Document
from docx.shared import RGBColor, Pt
from docx.enum.text import WD_COLOR_INDEX, WD_PARAGRAPH_ALIGNMENT
from random import choice, randint

def add_custom_comment(after_paragraph, text):
    """ Adds a custom-styled comment box right after a specified paragraph. """
    # Insert a new paragraph right after the current one
    # Since .insert_paragraph_before() inserts before, we add the comment to the start of the next paragraph
    comment_para = after_paragraph.insert_paragraph_before(text)

    # Set up the comment text and style
    run = comment_para.runs[0]
    run.font.color.rgb = RGBColor(0, 51, 102)  # Dark blue text
    run.bold = True
    run.font.highlight_color = WD_COLOR_INDEX.TURQUOISE  # Light blue highlight

    # Set paragraph formatting to blend with the document
    comment_para.style = after_paragraph.style
    comment_para.paragraph_format.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    comment_para.paragraph_format.space_before = Pt(0)
    comment_para.paragraph_format.space_after = Pt(0)

def apply_markup_to_paragraph(paragraph):
    """ Randomly apply highlight or strike-through to a given paragraph and optionally add a comment. """
    action = choice(['highlight', 'strike', 'comment', 'none'])

    # Normalize the paragraph text by removing leading/trailing whitespace and consecutive whitespace
    normalized_text = ' '.join(paragraph.text.split())

    if action == 'highlight':
        paragraph.clear()
        paragraph.add_run(normalized_text).font.highlight_color = WD_COLOR_INDEX.YELLOW
    elif action == 'strike':
        paragraph.clear()
        run = paragraph.add_run(normalized_text)
        run.font.strike = True
        run.font.color.rgb = RGBColor(255, 0, 0)

    if action == 'comment' and randint(0, 10) > 8:
        revised_text = ("Revised Clause: Reflecting the need for compliance with new regulations. "
                        "This includes a detailed review of all operational standards to ensure "
                        "they meet current legal requirements. Full compliance is essential to "
                        "mitigate potential risks and penalties associated with non-conformance.")
        add_custom_comment(paragraph, revised_text)

def process_document(input_path, output_path):
    document = Document(input_path)
    for paragraph in document.paragraphs:
        apply_markup_to_paragraph(paragraph)
    document.save(output_path)

# Usage
input_docx_path = 'playbook2.docx'
output_docx_path = 'answer1.docx'
process_document(input_docx_path, output_docx_path)

CONSIDERING MODEL'S OUTPUT AS INPUT TO GENERATE THE DOCX FILE WITH UPDATED TEXT


In [4]:
from docx import Document
from docx.shared import RGBColor, Pt
from docx.enum.text import WD_COLOR_INDEX, WD_PARAGRAPH_ALIGNMENT
import re

def normalize_text(input_text):
    """ Normalize spaces and strip text for better matching. """
    return re.sub(r'\s+', ' ', input_text.strip())

def add_custom_comment(after_paragraph, comment_text):
    """ Adds a custom-styled comment box right after a specified paragraph. """
    comment_para = after_paragraph.insert_paragraph_before()
    run = comment_para.add_run(comment_text)
    run.font.color.rgb = RGBColor(0, 51, 102)  # Dark blue text
    run.bold = True
    run.font.highlight_color = WD_COLOR_INDEX.TURQUOISE
    comment_para.paragraph_format.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    comment_para.paragraph_format.space_before = Pt(0)
    comment_para.paragraph_format.space_after = Pt(0)

def apply_model_outputs(document, model_outputs):
    for paragraph in document.paragraphs:
        normalized_paragraph_text = normalize_text(paragraph.text)
        for output in model_outputs:
            if normalize_text(output['text']) in normalized_paragraph_text:
                if output['action'] == 'highlight':
                    for run in paragraph.runs:
                        run.font.highlight_color = WD_COLOR_INDEX.YELLOW
                elif output['action'] == 'strike':
                    for run in paragraph.runs:
                        run.font.strike = True
                        run.font.color.rgb = RGBColor(255, 0, 0)
                if output['action'] == 'comment':
                    add_custom_comment(paragraph, output['comment'])

def process_document(input_path, output_path, model_outputs):
    document = Document(input_path)
    apply_model_outputs(document, model_outputs)
    document.save(output_path)

# Define model outputs
model_outputs = [
    {"action": "highlight", "text": "Service Provider will provide Customer the services selected in the Order Form", "comment": ""},
    {"action": "strike", "text": "THIS MASTER SERVICE AGREEMENT (COLLECTIVELY WITH THE PING ORDER", "comment": ""},
    {"action": "comment", "text": "Customer will pay Service Provider the Fees for the Services as listed on the applicable Order Form", "comment": "Review needed to adjust fees based on the new pricing model introduced in Q4.Review needed to adjust fees based on the new pricing model introduced in Q4.Review needed to adjust fees based on the new pricing model introduced in Q4.Review needed to adjust fees based on the new pricing model introduced in Q4."}
]

# Specify input and output paths
input_docx_path = 'playbook2.docx'
output_docx_path = 'answer2.docx'
process_document(input_docx_path, output_docx_path, model_outputs)


COMPARING THE TWO DOCX FILE

Lines beginning with - indicate text present in the first document but not in the second.


Lines beginning with + indicate text not present in the first document but added in the second.


Lines beginning with ? highlight potential areas of change (less common in plain text comparisons).

In [5]:
import difflib
from docx import Document

def read_docx(file_path):
    """Read text from a DOCX file and return two lists: normal text and struck-through text."""
    doc = Document(file_path)
    normal_text = []
    struck_text = []
    for paragraph in doc.paragraphs:
        if paragraph.text.strip():  # Check if paragraph is not just whitespace
            para_text = []
            for run in paragraph.runs:
                if run.font.strike:
                    struck_text.append(run.text.strip())
                else:
                    para_text.append(run.text.strip())
            if para_text:
                normal_text.append(' '.join(para_text))
    return normal_text, struck_text

def compare_documents(input_path, output_path):
    """Compare two DOCX files considering strike-through text as removed in the output."""
    input_text, _ = read_docx(input_path)
    output_text, output_struck_text = read_docx(output_path)
    
    # Combine output text and struck-through text as if the struck-through text is removed
    combined_output_text = output_text + output_struck_text

    # Create a Differ object
    differ = difflib.Differ()
    diff = list(differ.compare(input_text, combined_output_text))
    
    # Print the differences
    print("\nDifferences between documents:")
    for line in diff:
        if line.startswith('- ') or line.startswith('+ '):
            print(line)

# Usage
input_docx_path = 'playbook2.docx'
output_docx_path = 'answer2.docx'
compare_documents(input_docx_path, output_docx_path)



Differences between documents:
- THIS  MASTER  SERVICE  AGREEMENT  (COLLECTIVELY  WITH  THE  PING  ORDER FORM (THE  “ORDER FORM”)  AND  ALL OTHER  EXHIBITS  AND  ATTACHMENTS HERETO, THE “AGREEMENT”) WILL BE LEGALLY BINDING ON FIKE INC.
+ Review needed to adjust fees based on the new pricing model introduced in Q4.Review needed to adjust fees based on the new pricing model introduced in Q4.Review needed to adjust fees based on the new pricing model introduced in Q4.Review needed to adjust fees based on the new pricing model introduced in Q4.
+ THIS
+ 
+ MASTER
+ 
+ SERVICE
+ 
+ AGREEMENT
+ 
+ (COLLECTIVELY
+ 
+ WITH
+ 
+ THE
+ 
+ PING
+ 
+ ORDER FORM (THE
+ 
+ “ORDER FORM”)
+ 
+ AND
+ 
+ ALL OTHER
+ 
+ EXHIBITS
+ 
+ AND
+ 
+ ATTACHMENTS HERETO, THE “AGREEMENT”) WILL BE LEGALLY BINDING ON FIKE INC.
